In [17]:
import torch
import argparse
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [144]:
# Code from part 2 except the GUI stuff is removed
import random
from math import sqrt


class Point:
    x = -1
    y = -1

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return "(" + str(self.x) + ", " + str(self.y) + ")"

class Wire:
    start = Point(-1, -1)
    end = Point(-1, -1)

    def __init__(self, start, end):
        self.start = start
        self.end = end

    def __str__(self):
        return self.start.__str__() + self.end.__str__()

class Electrode:
    radius = 0
    center = Point(-1, -1)
    wires = []

    def __init__(self, radius, center):
        self.radius = radius
        self.center = center

    def __str__(self):
        return self.center.__str__()

def create_wire(l):
    '''
    Input: l, dimension of grid 
    Output: (Point1, Point2) which are start and end points of wire
    '''

    # pick unique ints q1, q2 from [0, 4]
    q1 = random.randint(0, 3)
    q2 = random.randint(0, 3)
    while q1 == q2:
        q2 = random.randint(0, 3)

    # generate random points
    x1 = random.uniform(0, l)
    y1 = random.uniform(0, l)
    x2 = random.uniform(0, l)
    y2 = random.uniform(0, l)
    
    # use q to "anchor" points to a wall using table 2
    if q1 == 0:
        y1 = 0
    elif q1 == 1:
        x1 = 0
    elif q1 == 2:
        y1 = l
    elif q1 == 3:
        x1 = l

    if q2 == 0:
        y2 = 0
    elif q2 == 1:
        x2 = 0
    elif q2 == 2:
        y2 = l
    elif q2 == 3:
        x2 = l

    return Wire(Point(x1, y1), Point(x2, y2))

def create_electrodes(n_e, l, r_e):
    '''
    Input:     n_e, number of electrodes
            l, dimension of grid
            r_e, radius of electrode

    Output: electrodes, list of electrodes
    '''
    electrodes = []

    # spacing in x and y direction between electrode centers
    spacing = l / (int(sqrt(n_e) + 1))

    x = spacing
    y = spacing

    while y < l:
        while x < l:
            center = Point(x, y)
            electrodes.append(Electrode(r_e, center))
            x += spacing
        y += spacing
        x = spacing

    # for e in electrodes:
    #     print(e)

    return electrodes


def electrode_touching_wire(electrode, wires):
    '''
    Input:     electrode, a singular Electrode object
            wires, a list of Wire objects

    Output:    touching_wires, a list of Wires touching the Electrode
    '''
    touching_wires = []
    for wire in wires:
        xe, ye = electrode.center.x, electrode.center.y
        x2, y2 = wire.end.x, wire.end.y
        x1, y1 = wire.start.x, wire.start.y

        numer = abs(xe * (y2 - y1) - ye * (x2 - x1) + x2 * y1 - y2 * x1)
        denom = sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

        if (numer / denom) <= electrode.radius:
            # print(numer / denom, electrode.radius)
            touching_wires.append(wire)

    return touching_wires

def adjacent_electrodes(e1, e2):
    '''
    Input:     e1, e2, Electrodes
    Output: touching, int that is 1 if e1 and e2 share a wire, 0 else
    '''
    wires1 = e1.wires
    wires2 = e2.wires

    for w1 in wires1:
        for w2 in wires2:

            start1 = w1.start
            start2 = w2.start

            end1 = w1.end
            end2 = w2.end

            equal_start = (start1.x == start2.x) and (start1.y == start2.y)
            equal_end = (end1.x == end2.x) and (end1.y == end2.y)

            if equal_start and equal_end:
                return 1

    return 0

def adj_matrix(a, r_e, n_e, density_constant):
    '''
    This is the main function that runs the simulation.

    Input:     a, spacing between electrodes
            r_e, electrode radius
            n_e, number of electrodes
            density_constant, denoted as lambda in the paper

    Output: adjacency_matrix, an adjacency matrix of electrodes
    '''

    l = a * (1 + sqrt(n_e))                    # dim of grid
    n_w = int(density_constant * sqrt(n_e))    # num of wires

    # Generate wires
    wires = [create_wire(l) for x in range(0, n_w)]
    
    # Generate electrodes
    electrodes = create_electrodes(n_e, l, r_e)

    # Generate list of wires touching electrodes
    for e in electrodes:
        e.wires = electrode_touching_wire(e, wires)

    # Generate adjacency list
    sqrt_ne = int(sqrt(n_e))
    adjacency_matrix = [[0] * n_e] * n_e
    
    for i in range(0, n_e):
        adjacency_matrix[i] = [0]*n_e
        for j in range(0, n_e):
            if i != j:
                electrode1 = electrodes[i]
                electrode2 = electrodes[j]

                adjacency_matrix[i][j] = adjacent_electrodes(electrode1, electrode2)

    for i in range(0, n_e):
        s = ""
        for j in range(0, n_e):
            s += str(adjacency_matrix[i][j]) + " "
        print(s)

    return adjacency_matrix
#     gui(adjacency_matrix, electrodes, wires, r_e, l)

# adjacency_matrix = adj_matrix(a=1, r_e=0.4, n_e=9, density_constant=30)







In [145]:
class Net(nn.Module):
    
    input_size = 784
    num_classes = 10
    
    p = 71
    q = 10
    
    n_w = 30
    
    A = adj_matrix(a=1, r_e=0.4, n_e=p+q, density_constant=30)
    
    def __init__(self):
        super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
#         self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
#         self.conv2_drop = nn.Dropout2d()
#         self.fc1 = nn.Linear(320, 50)
#         self.fc2 = nn.Linear(50, 10)
        self.layer_one = nn.Linear(784, self.p)
        self.layer_two = nn.Linear(self.p, self.n_w)
        self.layer_three = nn.Linear(self.n_w, self.q)
#         self.layer_three = nn.Softmax()


        
    def forward(self, x):
#         x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 784)
        x = self.layer_one(x)
        
        # try to update weights 
        
        layer_1_weights = list(self.layer_one.parameters())[0]
        print(len(layer_1_weights))
#         for i in range(len(layer_1_weights)):
#             print(layer_1_weights[i])
#             if layer_1_weights[i] < 0:
#                 print('negative weight!')
#                 layer_1_weights[i] = 0
            
        x = self.layer_two(x)
        x = self.layer_three(x)
        
#         return x
        return F.log_softmax(x, dim=1)

0 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 0 0 0 
1 0 1 1 1 0 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 0 1 0 1 1 1 1 0 0 1 0 1 1 1 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 
1 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 0 0 1 0 0 1 0 1 1 1 0 1 0 0 0 1 0 1 1 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 
0 1 1 0 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 1 1 0 0 0 1 0 
0 1 1 1 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0 1 0 0 0 1 1 1 1 1 0 1 0 1 1 0 1 1 1 0 0 1 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 1 1 0 1 0 0 1 0 1 0 1 1 1 0 0 1 0 1 0 1 
0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 1 0 1 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 
0 0 0 0 1 1 0 1 1 0 1 

In [146]:
def train(model, train_loader, optimizer, epoch):
    model.train() # training mode
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [147]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            criterion = nn.CrossEntropyLoss()
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [148]:
def main():
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=2450, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=2450, shuffle=True)


    model = Net()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

    for epoch in range(1, 10 + 1):
        train(model, train_loader, optimizer, epoch)
        test(model, test_loader)


In [ ]:
main()

71
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.368390
71
71
71
71
71
71
71
71
71
71
Train Epoch: 1 [24500/60000 (40%)]	Loss: 2.159819
71
71
71
71
71
71
71
71
71
71
Train Epoch: 1 [49000/60000 (80%)]	Loss: 1.947361
71
71
71
71
71
71
71
71
71

Test set: Average loss: 0.0009, Accuracy: 6176/10000 (62%)

71
Train Epoch: 2 [0/60000 (0%)]	Loss: 1.845804
71
71
71
71
71
71
71
71
71
71
Train Epoch: 2 [24500/60000 (40%)]	Loss: 1.647699
71
71
71
71
71
71
71
71
71
71
Train Epoch: 2 [49000/60000 (80%)]	Loss: 1.423628
71
71
71
71
71
71
71
71
71

Test set: Average loss: 0.0007, Accuracy: 7186/10000 (72%)

71
Train Epoch: 3 [0/60000 (0%)]	Loss: 1.352722
71
71
71
71
71
71
71
71
71
71
Train Epoch: 3 [24500/60000 (40%)]	Loss: 1.188235
71
71
71
71
71
71
71
71
71
71
Train Epoch: 3 [49000/60000 (80%)]	Loss: 1.038149
71
71
71
71
71
71
71
71
71

Test set: Average loss: 0.0005, Accuracy: 7858/10000 (79%)

71
Train Epoch: 4 [0/60000 (0%)]	Loss: 1.016999
71
